In [ ]:
# First dealing with all of the imports
import os
import cv2
import numpy as np
from tensorflow import keras as keras
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [ ]:
# Loading and Preprocessing the Dataset 
import xml.etree.ElementTree as ElemTree
# Images and labels list
images = []
labels = []

annotation_path = 'dataset1/annotations'
images_path = 'dataset1/images'

# Function to parse the xml file 
def parse_xml(xml_file):
    tree = ElemTree.parse(xml_file)
    root = tree.getroot()

    bounding_boxes = []
    temp_labels = []

    for object in root.findall('object'):
        temp_labels.append(object[0].text)
        xmin = int(object[5][0].text)
        ymin = int(object[5][1].text)
        xmax = int(object[5][2].text)
        ymax = int(object[5][3].text)
        bounding_boxes.append([xmin, ymin, xmax, ymax])

    return bounding_boxes, temp_labels

# Looping through the annotations file directory 
for xml_file in os.listdir(annotation_path):
    if xml_file.endswith(".xml"):
        image_path = os.path.join(images_path, xml_file[:-4] + ".png")
        xml_path = os.path.join(annotation_path, xml_file)

        image = cv2.imread(image_path)
        bounding_boxes, xml_labels = parse_xml(xml_path)

        for bound_box, lab in zip(bounding_boxes, xml_labels):
            xmin, ymin, xmax, ymax = bound_box
            roi = image[ymin:ymax, xmin:xmax]
            roi = cv2.resize(roi, (400, 400))
            roi = img_to_array(roi)
            roi = preprocess_input(roi)
            images.append(roi)
            labels.append(lab)

images = np.array(images, dtype="float32")
labels = np.array(labels)


In [ ]:
# Encoding the labels (with mask, no mask, wearing a mask incorrectly)
lb = LabelBinarizer()
labels = lb.fit_transform(labels).astype(int)

In [ ]:
print(images.shape)
print(labels.shape)
print(lb.classes_)

In [ ]:
# Splitting the dataset into training and testing sets
(train_images, test_images, train_labels, test_labels) = train_test_split(images, labels, test_size = 0.30, stratify=labels, random_state = 42)
# Test size of 0.20 means that 20% of data is for the test, and the rest is for the training set. 
# Picked a ranom state for reproduciablity if needed.

In [ ]:
# Data augmentation step.
# Not entirely needed but helps to increase the diversity of the dataset
data_aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [ ]:
train_data_gen = data_aug.flow(train_images, train_labels, batch_size=4)

In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

from keras.optimizers import Adam
from keras.losses import binary_focal_crossentropy



base_model = VGG16(weights='imagenet', include_top=False, input_shape=(400, 400, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Create full model 
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=binary_focal_crossentropy,
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import LearningRateScheduler
import math

# Step decay schedule 
def step_decay(epoch):
   initial_lrate = 0.1
   drop = 0.5
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(step_decay)

callbacks=[lr_scheduler]

model.fit(
  train_images, 
  train_labels,
  epochs=50,
  batch_size=2,
  validation_data=(test_images, test_labels),
  callbacks=[lr_scheduler]
)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
model.save('model4')

In [ ]:
import cv2
from keras.models import load_model
import numpy as np

# Load models
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
mask_model = load_model('model4')

# Colors for boxes
colors = [(0,255,0),(0,0,255),(255,0,0)] 

cap = cv2.VideoCapture(0)

while True:
  ret, frame = cap.read()

  faces = face_cascade.detectMultiScale(frame)

  for (x,y,w,h) in faces:

    # Preprocess face
    face_img = frame[y:y+h, x:x+w] 
    face_img = cv2.resize(face_img, (224,224))
    face_img = preprocess(face_img)  

    # Predict
    preds = mask_model.predict(face_img)[0]
    print(preds)
    pred_class = np.argmax(preds)
    print(pred_class)
    pred_label = lb.inverse_transform()


    # Draw box
    color = colors[pred_class]
    cv2.rectangle(frame, (x,y), (x+w,y+h), color, 2)

    # Annotate
    label = f"Prediction: {pred_label}"
    cv2.putText(frame, label, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  cv2.imshow('Video', frame)

  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()  
cv2.destroyAllWindows()
